In [0]:
from pyspark.sql import functions as f
from delta.tables import DeltaTable

In [0]:
%run /Workspace/Users/nikamswapnalee111@gmail.com/Consolidated_pipeline/Consolidated_pipeline/setup/utilities

In [0]:
print(bronze_schema, silver_schema, gold_schema)

bronze silver gold


In [0]:
# This code uses Databricks widgets to take user input and dynamically build an S3 path so the same notebook can process different datasets without changing the code.

# Creates input boxes at top of this notebook
dbutils.widgets.text("catalog", "fmcg", "Catalog")
dbutils.widgets.text("data_source", "customers", "Data Source")

# Reading what user typed
catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

# Building the S3 path dynamically  - Read all CSV files from that folder
base_path = f's3://sportsbar-svn/{data_source}/*.csv'
# Printing the final path
print(base_path)

s3://sportsbar-svn/customers/*.csv


# **Bronze Layer**

**### **To move data from S3 to the Bronze layer, I read the raw CSV files from S3 using Spark, added basic ingestion metadata like read timestamp and source file details, and then wrote the data as a Delta table in the Bronze layer**.**

In [0]:
df = (
    spark.read.format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(base_path)
        .withColumn("read_timestamp", f.current_timestamp())
        .select("*", "_metadata.file_name", "_metadata.file_size")
)

In [0]:
# print check data type
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = false)
 |-- file_name: string (nullable = false)
 |-- file_size: long (nullable = false)



In [0]:
display(df.limit(10))

customer_id,customer_name,city,read_timestamp,file_name,file_size
789201,FitFuel Market,Bengaluru,2026-02-01T10:43:27.619Z,customers.csv,1404
789202,FitFuel Market,Hyderabad,2026-02-01T10:43:27.619Z,customers.csv,1404
789203,FitFuel Market,New Delhi,2026-02-01T10:43:27.619Z,customers.csv,1404
789301,Athlete's Choice Store,Bengaluru,2026-02-01T10:43:27.619Z,customers.csv,1404
789303,Athlete's Choice Store,New Delhi,2026-02-01T10:43:27.619Z,customers.csv,1404
789101,Endurance Foods,Bengalore,2026-02-01T10:43:27.619Z,customers.csv,1404
789102,Endurance Foods,Hyderabad,2026-02-01T10:43:27.619Z,customers.csv,1404
789103,Endurance Foods,New Delhi,2026-02-01T10:43:27.619Z,customers.csv,1404
789121,HydroBoost Nutrition,Hyderabad,2026-02-01T10:43:27.619Z,customers.csv,1404
789122,HydroBoost Nutrition,New Delhi,2026-02-01T10:43:27.619Z,customers.csv,1404


In [0]:
df.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

# Silver Processing**
# **

**reads customer data from the Bronze layer, cleans and standardizes it, fixes data quality issues, and writes a clean Silver table.**

I read customer data from the Bronze layer, identified and removed duplicate customers using customer_id, cleaned customer names and city values, fixed city typos using mapping logic, handled nulls, and applied business-approved corrections. I standardized the data format, added business metadata columns, and finally wrote a clean, analytics-ready Silver Delta table with Change Data Feed enabled.”

In [0]:
# Read the data from bronze table
df_bronze = spark.sql(f"SELECT * FROM {catalog}.{bronze_schema}.{data_source};")
# View sample data
df_bronze.show(10)

+-----------+--------------------+---------+--------------------+-------------+---------+
|customer_id|       customer_name|     city|      read_timestamp|    file_name|file_size|
+-----------+--------------------+---------+--------------------+-------------+---------+
|     789201|      FitFuel Market|Bengaluru|2026-02-01 10:43:...|customers.csv|     1404|
|     789202|      FitFuel Market|Hyderabad|2026-02-01 10:43:...|customers.csv|     1404|
|     789203|      FitFuel Market|New Delhi|2026-02-01 10:43:...|customers.csv|     1404|
|     789301|Athlete's Choice ...|Bengaluru|2026-02-01 10:43:...|customers.csv|     1404|
|     789303|Athlete's Choice ...|New Delhi|2026-02-01 10:43:...|customers.csv|     1404|
|     789101|     Endurance Foods|Bengalore|2026-02-01 10:43:...|customers.csv|     1404|
|     789102|     Endurance Foods|Hyderabad|2026-02-01 10:43:...|customers.csv|     1404|
|     789103|     Endurance Foods|New Delhi|2026-02-01 10:43:...|customers.csv|     1404|
|     7891

In [0]:
# check column name and data types
df_bronze.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = true)
 |-- file_name: string (nullable = true)
 |-- file_size: long (nullable = true)



Transformations


1. Identify and Remove Duplicates (customers - customer_id)

In [0]:
# Remove Duplicates
# Identify duplicate customers
df_duplicates = df_bronze.groupBy("customer_id").count().filter(f.col("count") > 1)
# print those  records - finds those records based on customer_id
display(df_duplicates)

customer_id,count
789321,2
789503,2
789522,2
789603,2


In [0]:
# Count before cleaning
print('Rows before duplicates dropped: ', df_bronze.count())

# Remove duplicate customers - Keeps only unique customers
df_silver = df_bronze.dropDuplicates(['customer_id'])
# Confirms that no duplicates remain
print('Rows after duplicates dropped: ', df_silver.count())

Rows before duplicates dropped:  39
Rows after duplicates dropped:  35


2. Clean customer names - remove spaces

In [0]:
# check those values : Trim spaces in custromer_name
# Finds names with extra spaces
display(
    df_silver.filter(f.col("customer_name") != f.trim(f.col("customer_name")))
)

customer_id,customer_name,city,read_timestamp,file_name,file_size
789121,HydroBoost Nutrition,Hyderabad,2026-02-01T10:43:49.221Z,customers.csv,1404
789401,SprintX nutrition,Bengaluru,2026-02-01T10:43:49.221Z,customers.csv,1404
789420,ZenAthlete foods,null,2026-02-01T10:43:49.221Z,customers.csv,1404
789421,ZenAthlete Foods,Hyderbad,2026-02-01T10:43:49.221Z,customers.csv,1404
789521,PrimeFuel Nutrition,null,2026-02-01T10:43:49.221Z,customers.csv,1404
789702,StaminaX Store,Hyderabad,2026-02-01T10:43:49.221Z,customers.csv,1404


In [0]:
# remove those trim values
# Removes unwanted leading/trailing spaces

df_silver = df_silver.withColumn(
    "customer_name",
    f.trim(f.col("customer_name"))
)

In [0]:
# # Sanity Check

# # check those values
# display(
 #   df_silver.filter(f.col("customer_name") != f.trim(f.col("customer_name")))
#)


3 Data Quality Fix : Correcting City name Typos

In [0]:
# Check unique cities - that will reveals typos & inconsistant city names
df_silver.select('city').distinct().show()

+----------+
|      city|
+----------+
| Bengaluru|
| Hyderabad|
| New Delhi|
| Bengalore|
|Hyderabadd|
|      NULL|
|  Hyderbad|
| NewDelhee|
|  NewDelhi|
|Bengaluruu|
|  NewDheli|
+----------+



In [0]:
# Fix city name typos
# # typo dictionary
# city_typos = {
#     'Bengaluru': ['Bengaluruu', 'Bengaluruu', 'Bengalore'],
#     'Hyderabad': ['Hyderabadd', 'Hyderbad'],
#     'New Delhi': ['NewDelhi', 'NewDheli', 'NewDelhee']
# }

# Dictonary to correct misspellings
# typos → correct names
city_mapping = {
    'Bengaluruu': 'Bengaluru',
    'Bengalore': 'Bengaluru',

    'Hyderabadd': 'Hyderabad',
    'Hyderbad': 'Hyderabad',

    'NewDelhi': 'New Delhi',
    'NewDheli': 'New Delhi',
    'NewDelhee': 'New Delhi'
}

# Only valid citites allowed
allowed = ["Bengaluru", "Hyderabad", "New Delhi"]

# Fix typos & invalid city - null
df_silver = (
    df_silver
    .replace(city_mapping, subset=["city"])
    .withColumn(
        "city",
        f.when(f.col("city").isNull(), None)
         .when(f.col("city").isin(allowed), f.col("city"))
         .otherwise(None)
    )
)

In [0]:
# Sanity check
df_silver.select('city').distinct().show()

+---------+
|     city|
+---------+
|Bengaluru|
|Hyderabad|
|New Delhi|
|     NULL|
+---------+



4. Standardize customer names -  Fix Title-Casting Issue

In [0]:
df_silver.select('customer_name').distinct().show()

+--------------------+
|       customer_name|
+--------------------+
|      Fitfuel Market|
|Athlete's Choice ...|
|     Endurance Foods|
|Hydroboost Nutrition|
|Macrobite Superfoods|
|      Powersnack Hub|
|   Sprintx Nutrition|
|    Zenathlete Foods|
|Peak Performance ...|
| Primefuel Nutrition|
|       Recovery Lane|
|      Staminax Store|
|Eliteathlete Nutr...|
|      Gameplan Foods|
|   Champion's Choice|
+--------------------+



In [0]:
# Title case fix - converts names to title case () - Each words first letter is capital
df_silver = df_silver.withColumn(
    "customer_name",
    f.when(f.col("customer_name").isNull(), None)
     .otherwise(f.initcap("customer_name"))
)

In [0]:
# sanity check

df_silver.select('customer_name').distinct().show()

+--------------------+
|       customer_name|
+--------------------+
|      Fitfuel Market|
|Athlete's Choice ...|
|     Endurance Foods|
|Hydroboost Nutrition|
|Macrobite Superfoods|
|      Powersnack Hub|
|   Sprintx Nutrition|
|    Zenathlete Foods|
|Peak Performance ...|
| Primefuel Nutrition|
|       Recovery Lane|
|      Staminax Store|
|Eliteathlete Nutr...|
|      Gameplan Foods|
|   Champion's Choice|
+--------------------+



5. Handling Missing Cities - Find records with missing city

In [0]:
df_silver.filter(f.col("city").isNull()).show(truncate=False)

+-----------+-------------------+----+--------------------------+-------------+---------+
|customer_id|customer_name      |city|read_timestamp            |file_name    |file_size|
+-----------+-------------------+----+--------------------------+-------------+---------+
|789403     |Sprintx Nutrition  |NULL|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789420     |Zenathlete Foods   |NULL|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789521     |Primefuel Nutrition|NULL|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789603     |Recovery Lane      |NULL|2026-02-01 10:43:49.221584|customers.csv|1404     |
+-----------+-------------------+----+--------------------------+-------------+---------+



In [0]:
null_customer_names = ['Sprintx Nutrition', 'Zenathlete Foods', 'Primefuel Nutrition', 'Recovery Lane']
df_silver.filter(f.col("customer_name").isin(null_customer_names)).show(truncate=False)

+-----------+-------------------+---------+--------------------------+-------------+---------+
|customer_id|customer_name      |city     |read_timestamp            |file_name    |file_size|
+-----------+-------------------+---------+--------------------------+-------------+---------+
|789401     |Sprintx Nutrition  |Bengaluru|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789402     |Sprintx Nutrition  |Hyderabad|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789403     |Sprintx Nutrition  |NULL     |2026-02-01 10:43:49.221584|customers.csv|1404     |
|789420     |Zenathlete Foods   |NULL     |2026-02-01 10:43:49.221584|customers.csv|1404     |
|789421     |Zenathlete Foods   |Hyderabad|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789422     |Zenathlete Foods   |New Delhi|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789520     |Primefuel Nutrition|Bengaluru|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789521     |Primefuel Nutrition|NULL     |2026-02

 6. Business - approved city fixes

In [0]:
# Business Confirmation Note: City corrections confirmed by business team
# Manuly confirmed by business team
customer_city_fix = {
    # Sprintx Nutrition
    789403: "New Delhi",

    # Zenathlete Foods
    789420: "Bengaluru",

    # Primefuel Nutrition
    789521: "Hyderabad",

    # Recovery Lane
    789603: "Hyderabad"
}

# Convert dictionary to Spark DataFrame
df_fix = spark.createDataFrame(
    [(k, v) for k, v in customer_city_fix.items()],
    ["customer_id", "fixed_city"]
)

display(df_fix)

customer_id,fixed_city
789403,New Delhi
789420,Bengaluru
789521,Hyderabad
789603,Hyderabad


In [0]:
# if city was null - replace with confiremed city
df_silver = (
    df_silver
    .join(df_fix, "customer_id", "left")
    .withColumn(
        "city",
        f.coalesce("city", "fixed_city")   # Replace null with fixed city
    )
    .drop("fixed_city")
)

In [0]:
# Sanity Checks

null_customer_names = ['Sprintx Nutrition', 'Zenathlete Foods', 'Primefuel Nutrition', 'Recovery Lane']
df_silver.filter(f.col("customer_name").isin(null_customer_names)).show(truncate=False)

+-----------+-------------------+---------+--------------------------+-------------+---------+
|customer_id|customer_name      |city     |read_timestamp            |file_name    |file_size|
+-----------+-------------------+---------+--------------------------+-------------+---------+
|789420     |Zenathlete Foods   |Bengaluru|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789601     |Recovery Lane      |Bengaluru|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789402     |Sprintx Nutrition  |Hyderabad|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789603     |Recovery Lane      |Hyderabad|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789401     |Sprintx Nutrition  |Bengaluru|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789421     |Zenathlete Foods   |Hyderabad|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789520     |Primefuel Nutrition|Bengaluru|2026-02-01 10:43:49.221584|customers.csv|1404     |
|789522     |Primefuel Nutrition|New Delhi|2026-02

7. Convert customer_id to string

In [0]:
# Change data type - Makes customer_id to string
df_silver = df_silver.withColumn("customer_id", f.col("customer_id").cast("string"))
print(df_silver.printSchema())

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = true)
 |-- file_name: string (nullable = true)
 |-- file_size: long (nullable = true)

None


8. Standardizing Customer Attributes to Match Parent Company Data Model 
(Parent +Child customer table combined + Metadata like real time stamp, file name & size)

In [0]:
# Create final business-ready columns

df_silver = (
    df_silver
    # Build final customer column: "CustomerName-City" or "CustomerName-Unknown"
    .withColumn(
        "customer",
        f.concat_ws("-", "customer_name", f.coalesce(f.col("city"), f.lit("Unknown")))
    )
    
    # Static attributes aligned with parent data model
    .withColumn("market", f.lit("India"))
    .withColumn("platform", f.lit("Sports Bar"))
    .withColumn("channel", f.lit("Acquisition"))
)

In [0]:
display(df_silver.limit(5))

customer_id,customer_name,city,read_timestamp,file_name,file_size,customer,market,platform,channel
789503,Peak Performance Store,New Delhi,2026-02-01T10:43:49.221Z,customers.csv,1404,Peak Performance Store-New Delhi,India,Sports Bar,Acquisition
789420,Zenathlete Foods,Bengaluru,2026-02-01T10:43:49.221Z,customers.csv,1404,Zenathlete Foods-Bengaluru,India,Sports Bar,Acquisition
789703,Staminax Store,New Delhi,2026-02-01T10:43:49.221Z,customers.csv,1404,Staminax Store-New Delhi,India,Sports Bar,Acquisition
789621,Eliteathlete Nutrition,Hyderabad,2026-02-01T10:43:49.221Z,customers.csv,1404,Eliteathlete Nutrition-Hyderabad,India,Sports Bar,Acquisition
789101,Endurance Foods,Bengaluru,2026-02-01T10:43:49.221Z,customers.csv,1404,Endurance Foods-Bengaluru,India,Sports Bar,Acquisition


In [0]:
# write to silver delta table - Saves clean, standardized data to silver layer

df_silver.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .option("mergeSchema", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{silver_schema}.{data_source}")

# **Gold**

In this pipeline, I read cleaned data from the Silver layer, select only business-required customer attributes, and write it to the Gold layer as a Delta dimension table. I then use Delta Lake merge (upsert) logic to update existing customer records and insert new ones, ensuring the Gold table always contains the latest customer information.”

In [0]:
# read data from silver layer - spark dataframe
df_silver = spark.sql(f"SELECT * FROM {catalog}.{silver_schema}.{data_source};")


# Select only required columns (busieness ready data)
# "customer_id, customer_name, city, read_timestamp, file_name, file_size, customer, market, platform, channel"
df_gold = df_silver.select("customer_id", "customer_name", "city", "customer", "market", "platform", "channel")

In [0]:
# write data to gold layer as delta table
df_gold.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{gold_schema}.sb_dim_{data_source}")

In [0]:
# Reference the gold Delta table - this table will be updated incrementally
delta_table = DeltaTable.forName(spark, "fmcg.gold.dim_customers")

# Prepare source data for merge(upsert) - reading new customer data
df_child_customers = spark.table("fmcg.gold.sb_dim_customers").select(
   # Renaming customer_id to customer_code
    f.col("customer_id").alias("customer_code"),
    "customer",
    "market",
    "platform",
    "channel"
)

In [0]:
# Merge into gold table - upsert opration
delta_table.alias("target").merge(
    source=df_child_customers.alias("source"),
    condition="target.customer_code = source.customer_code"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]